##### Sentiment Analysis
This notebook leverages a NLP model pre-trained on portugese/brazil reviews.

In [43]:
# Libraries setup
import os
import re
import string
import nltk

import pandas as pd
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# # Download the WordNet resource
# nltk.download('wordnet')
nltk.download('stopwords')

In [44]:
# Define paths
fpath_data = os.environ.get("FPATH_DATA")

# Load reviews data
reviews = pd.read_csv(fpath_data+"order_reviews.csv")
reviews

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53
...,...,...,...,...,...,...,...
99995,f3897127253a9592a73be9bdfdf4ed7a,22ec9f0669f784db00fa86d035cf8602,5,NaN,NaN,2017-12-09 00:00:00,2017-12-11 20:06:42
99996,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,5,NaN,"Excelente mochila, entrega super rápida. Super...",2018-03-22 00:00:00,2018-03-23 09:10:43
99997,1adeb9d84d72fe4e337617733eb85149,7725825d039fc1f0ceb7635e3f7d9206,4,NaN,NaN,2018-07-01 00:00:00,2018-07-02 12:59:13
99998,be360f18f5df1e0541061c87021e6d93,f8bd3f2000c28c5342fedeb5e50f2e75,1,NaN,Solicitei a compra de uma capa de retrovisor c...,2017-12-15 00:00:00,2017-12-16 01:29:43


In [45]:
reviews[reviews['review_comment_title'].notnull()]

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
9,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,recomendo,aparelho eficiente. no site a marca do aparelh...,2018-05-22 00:00:00,2018-05-23 16:45:47
15,3948b09f7c818e2d86c9a546758b2335,e51478e7e277a83743b6f9991dbfa3fb,5,Super recomendo,"Vendedor confiável, produto ok e entrega antes...",2018-05-23 00:00:00,2018-05-24 03:00:01
19,373cbeecea8286a2b66c97b1b157ec46,583174fbe37d3d5f0d6661be3aad1786,1,Não chegou meu produto,Péssimo,2018-08-15 00:00:00,2018-08-15 04:10:37
22,d21bbc789670eab777d27372ab9094cc,4fc44d78867142c627497b60a7e0228a,5,Ótimo,Loja nota 10,2018-07-10 00:00:00,2018-07-11 14:10:25
34,c92cdd7dd544a01aa35137f901669cdf,37e7875cdce5a9e5b3a692971f370151,4,Muito bom.,Recebi exatamente o que esperava. As demais en...,2018-06-07 00:00:00,2018-06-09 18:44:02
...,...,...,...,...,...,...,...
99967,0e7bc73fde6782891898ea71443f9904,bd78f91afbb1ecbc6124974c5e813043,4,👍,Aprovado!,2018-07-04 00:00:00,2018-07-05 00:25:13
99971,58be140ccdc12e8908ff7fd2ba5c7cb0,0ebf8e35b9807ee2d717922d5663ccdb,5,muito bom produto,"Ficamos muito satisfeitos com o produto, atend...",2018-06-30 00:00:00,2018-07-02 23:09:35
99972,51de4e06a6b701cb2be47ea0e689437b,b7467ae483dbe956fe9acdf0b1e6e3f4,3,Não foi entregue o pedido,Bom dia \r\nDas 6 unidades compradas só recebi...,2018-06-05 00:00:00,2018-06-06 10:52:19
99974,40743b46a0ee86375cedb95e82b78d75,3e93213bb8fdda91186b4018b2fe0030,5,OTIMA EMBALAGEM,NaN,2018-08-08 00:00:00,2018-08-08 16:56:16


In [46]:
reviews[reviews['review_comment_message'].notnull()]

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53
9,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,recomendo,aparelho eficiente. no site a marca do aparelh...,2018-05-22 00:00:00,2018-05-23 16:45:47
12,4b49719c8a200003f700d3d986ea1a19,9d6f15f95d01e79bd1349cc208361f09,4,NaN,"Mas um pouco ,travando...pelo valor ta Boa.\r\n",2018-02-16 00:00:00,2018-02-20 10:52:22
15,3948b09f7c818e2d86c9a546758b2335,e51478e7e277a83743b6f9991dbfa3fb,5,Super recomendo,"Vendedor confiável, produto ok e entrega antes...",2018-05-23 00:00:00,2018-05-24 03:00:01
...,...,...,...,...,...,...,...
99983,df5fae90e85354241d5d64a8955b2b09,509b86c65fe4e2ad5b96408cfef9755e,5,NaN,Entregou dentro do prazo. O produto chegou em ...,2018-02-07 00:00:00,2018-02-19 19:47:23
99990,a709d176f59bc3af77f4149c96bae357,d5cb12269711bd1eaf7eed8fd32a7c95,3,NaN,"O produto não foi enviado com NF, não existe v...",2018-05-19 00:00:00,2018-05-20 21:51:06
99996,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,5,NaN,"Excelente mochila, entrega super rápida. Super...",2018-03-22 00:00:00,2018-03-23 09:10:43
99998,be360f18f5df1e0541061c87021e6d93,f8bd3f2000c28c5342fedeb5e50f2e75,1,NaN,Solicitei a compra de uma capa de retrovisor c...,2017-12-15 00:00:00,2017-12-16 01:29:43


In [47]:
reviews.isnull().sum()

review_id                      0
order_id                       0
review_score                   0
review_comment_title       88285
review_comment_message     58247
review_creation_date           0
review_answer_timestamp        0
dtype: int64

In [48]:
# It looks like all reviews are from Brazil according to customers.csv

# For the sake of simplicit, lets merge review comment title and message
reviews['review_concat'] = reviews['review_comment_title'].fillna('')+', '+reviews['review_comment_message'].fillna('')

# # Drop rows with null ['review_concat']
# reviews = reviews.dropna(subset=['review_concat'])

# Drop rows with only ', ' string
reviews = reviews[reviews.review_concat != ', ']

# Lets keep only relevant columns
reviews = reviews[['review_id', 'order_id', 'review_concat']]

In [49]:
reviews

,review_id,order_id,review_concat
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,", Recebi bem antes do prazo estipulado."
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,", Parabéns lojas lannister adorei comprar pela..."
9,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,"recomendo, aparelho eficiente. no site a marca..."
12,4b49719c8a200003f700d3d986ea1a19,9d6f15f95d01e79bd1349cc208361f09,", Mas um pouco ,travando...pelo valor ta Boa.\r\n"
15,3948b09f7c818e2d86c9a546758b2335,e51478e7e277a83743b6f9991dbfa3fb,"Super recomendo, Vendedor confiável, produto o..."
...,...,...,...
99983,df5fae90e85354241d5d64a8955b2b09,509b86c65fe4e2ad5b96408cfef9755e,", Entregou dentro do prazo. O produto chegou e..."
99990,a709d176f59bc3af77f4149c96bae357,d5cb12269711bd1eaf7eed8fd32a7c95,", O produto não foi enviado com NF, não existe..."
99996,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,", Excelente mochila, entrega super rápida. Sup..."
99998,be360f18f5df1e0541061c87021e6d93,f8bd3f2000c28c5342fedeb5e50f2e75,", Solicitei a compra de uma capa de retrovisor..."


##### Preprocessing

In [53]:
# Lets check if there are any numbers in the reviews - this will be helpfull for the second task
def search_numbers(text):
    return bool(re.search(r'\d+', text))

reviews['contain_num'] = reviews['review_concat'].apply(search_numbers)
reviews[reviews['contain_num']==True].head(50)

,review_id,order_id,review_concat,contain_num
9,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,"recomendo, aparelho eficiente. no site a marca...",True
22,d21bbc789670eab777d27372ab9094cc,4fc44d78867142c627497b60a7e0228a,"Ótimo, Loja nota 10",True
51,e233e51d11511bf30e568c76360ace52,548df2c6e5f089574614894bca78acf5,", recebi somente 1 controle Midea Split ESTILO...",True
115,ae728c1061bf163b4bd256ad9ee0bb83,45c780334bc32cb77559a65c5f171160,", Este foi o pedido\r\nBalde Com 128 Peças - B...",True
166,14613df954694f243dedeaa2c8a3a5c2,1e91914c9706a5d4665b8c5a6294b107,"4 , Se fosse vidro tinha quebrado; veio na c...",True
168,d7ea05bb4f9c77d606fd90b1a27e317d,bb5b0d8f3e9c964760c674f2aa2ae0cf,", Comprei o produto dia 25 de fevereiro e hoje...",True
169,185cb85ede91009d7a8e685687c037ee,2e5dc86c8c4aa663549caf5e31de840d,", fiz minha compra faz 30 dias e não recebi ai...",True
190,c40a6b6e0181e5ec0d12cbc2e12c49d3,f3468d6a2c7586711e9952242898aad4,"falta de produto e quebra, Faltou 1 produto e ...",True
207,6b341682ab39af9fa00d72d7388c903b,fd65651f855608ad1aa52ff4a741c31a,"Nota3, Comprar um produto correto na capa mas ...",True
212,557ae9c16a8ea3909db2fc5a01bc5151,287b045cfe9ba0ed08308efc06c366ae,"Suspense, Fiz um pedido de 4 garrafas de azeit...",True


In [ ]:
def remove_emojis(text):
    """Remove emojis from the text."""
    if isinstance(text, str):
        emoji_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                   u"\U00002500-\U00002BEF"  # chinese char
                                   u"\U00002702-\U000027B0"
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   u"\U0001f926-\U0001f937"
                                   u"\U00010000-\U0010ffff"
                                   u"\u2640-\u2642"
                                   u"\u2600-\u2B55"
                                   u"\u200d"
                                   u"\u23cf"
                                   u"\u23e9"
                                   u"\u231a"
                                   u"\ufe0f"  # dingbats
                                   u"\u3030"
                                   "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r'', text)
    else:
        return text

In [ ]:
def remove_stopwords(tokens):
    """Eliminate common stopwords from the tokenized text.
    Stop words are commonly used words like "the," "is," or 
    "and" that don't carry much meaning and can be removed to reduce noise in the data
    """
    # stop_words = set(stopwords.words('english')) # English version - we will anyway use PT as the translation will be done after preprocessing
    stop_words = set(stopwords.words('english')) # PT version
    return [word for word in tokens if word not in stop_words]

In [ ]:
def tokenize_text(text):
    """Split the text into individual words or tokens."""
    return word_tokenize(text)

In [ ]:
def remove_numbers(text):
    """Exclude numerical digits from the text."""
    return re.sub(r'\d+', '', text)

def remove_punctuation(text):
    """Remove punctuation marks from the text."""
    return text.translate(str.maketrans('', '', string.punctuation))

def remove_extra_whitespaces(text):
    """Remove extra white space from text."""
    return re.sub(r'\s+', ' ', text, flags=re.I)

def remove_single_chars(text):
    """Remove all single characters from text"""
    return re.sub(r'\s+[a-zA-Z]\s+', ' ', text)

def remove_special_chars(text):
    """Remove all the special characters from text."""
    return re.sub(r'\W', ' ', text)

def remove_not_alphabetical(text):
    """Remove any character that isn't alphabetical."""
    return re.sub(r'[^a-zA-Z\s]', '', text)

In [51]:
def preprocess(text):
    text = str(text).lower() # To lower
    text = remove_extra_whitespaces(text)
    text = remove_numbers(text)
    text = remove_punctuation(text)
    text = tokenize_text(text)
    text = remove_stopwords(text)
    text = remove_emojis(text)
    return text